In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import Callable, Optional

from anndata import AnnData

from geome.iterables import ToCategoryIterable
from geome.transforms import Categorize  


from geome.adata2data import AnnData2DataDefault
import pytorch_lightning as pl
import torch
import anndata as ad
from geome import transforms
from geome.adata2data import AnnData2DataByCategory
from utils.datasets import DatasetHartmann
from utils.models.non_linear_ncem import NonLinearNCEM
from geome.datamodule import GraphAnnDataModule



/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/spatialdata/__init__.py:9: UserWarning: Geopandas was set to use PyGEOS, changing to shapely 2.0 with:

	geopandas.options.use_pygeos = True

If you intended to use PyGEOS, set the option to False.
  _check_geopandas_using_shapely()


In [3]:


category = 'point'
to_iterable = ToCategoryIterable(category, axis='obs')

In [4]:
#Mibitof
dataset = DatasetHartmann(data_path='./example_data/hartmann/')
adatas = list(dataset.img_celldata.values())

# Merge the list of adatas and convert some string to categories as they should be
adata = ad.concat(adatas)

adata = Categorize(['donor', 'Cluster_preprocessed', 'point'], axis='obs')(adata) 

Loading data from raw files
registering celldata


/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


collecting image-wise celldata
adding graph-level covariates
Loaded 58 images with complete data from 4 patients over 63747 cells with 36 cell features and 8 distinct celltypes.


In [6]:
adata.obs['Cluster'].cat.categories

Index(['Endothelial', 'Epithelial', 'Fibroblast', 'Imm_other', 'Myeloid_CD11c',
       'Myeloid_CD68', 'Tcell_CD4', 'Tcell_CD8'],
      dtype='object')

In [7]:
split_adatas = list(to_iterable(adata))
split_adatas

               point cell_id cell_size donor       Cluster   
59191  scMEP_point_1       3     139.0  90de     Imm_other  \
59192  scMEP_point_1       4     230.0  90de     Tcell_CD4   
59193  scMEP_point_1       6     116.0  90de     Imm_other   
59194  scMEP_point_1       7    1082.0  90de     Imm_other   
59195  scMEP_point_1       9     649.0  90de     Imm_other   
...              ...     ...       ...   ...           ...   
18510  scMEP_point_9     835     579.0  90de     Imm_other   
18511  scMEP_point_9     836     167.0  90de     Imm_other   
18512  scMEP_point_9     837     274.0  90de     Imm_other   
18513  scMEP_point_9     839     250.0  90de     Imm_other   
18514  scMEP_point_9     840     170.0  90de  Myeloid_CD68   

      Cluster_preprocessed  
59191   Other immune cells  
59192          CD4 T cells  
59193   Other immune cells  
59194   Other immune cells  
59195   Other immune cells  
...                    ...  
18510   Other immune cells  
18511   Other immune ce

[AnnData object with n_obs × n_vars = 1338 × 36
     obs: 'point', 'cell_id', 'cell_size', 'donor', 'Cluster', 'Cluster_preprocessed'
     obsm: 'spatial', 'node_types',
 AnnData object with n_obs × n_vars = 311 × 36
     obs: 'point', 'cell_id', 'cell_size', 'donor', 'Cluster', 'Cluster_preprocessed'
     obsm: 'spatial', 'node_types',
 AnnData object with n_obs × n_vars = 768 × 36
     obs: 'point', 'cell_id', 'cell_size', 'donor', 'Cluster', 'Cluster_preprocessed'
     obsm: 'spatial', 'node_types',
 AnnData object with n_obs × n_vars = 1020 × 36
     obs: 'point', 'cell_id', 'cell_size', 'donor', 'Cluster', 'Cluster_preprocessed'
     obsm: 'spatial', 'node_types',
 AnnData object with n_obs × n_vars = 2100 × 36
     obs: 'point', 'cell_id', 'cell_size', 'donor', 'Cluster', 'Cluster_preprocessed'
     obsm: 'spatial', 'node_types',
 AnnData object with n_obs × n_vars = 1325 × 36
     obs: 'point', 'cell_id', 'cell_size', 'donor', 'Cluster', 'Cluster_preprocessed'
     obsm: 'spatia

In [8]:
split_adatas[0].obs['Cluster']

59191     Imm_other
59192     Tcell_CD4
59193     Imm_other
59194     Imm_other
59195     Imm_other
            ...    
60524     Imm_other
60525     Imm_other
60526    Fibroblast
60527     Imm_other
60528     Imm_other
Name: Cluster, Length: 1338, dtype: category
Categories (8, object): ['Endothelial', 'Epithelial', 'Fibroblast', 'Imm_other', 'Myeloid_CD11c', 'Myeloid_CD68', 'Tcell_CD4', 'Tcell_CD8']

In [30]:
split_adatas[0].obs['Cluster'].cat.set_categories(adata.obs['Cluster'].cat.categories)

59191     Imm_other
59192     Tcell_CD4
59193     Imm_other
59194     Imm_other
59195     Imm_other
            ...    
60524     Imm_other
60525     Imm_other
60526    Fibroblast
60527     Imm_other
60528     Imm_other
Name: Cluster, Length: 1338, dtype: category
Categories (8, object): ['Endothelial', 'Epithelial', 'Fibroblast', 'Imm_other', 'Myeloid_CD11c', 'Myeloid_CD68', 'Tcell_CD4', 'Tcell_CD8']

In [31]:
split_adatas[0].obs['Cluster']

59191     Imm_other
59192     Tcell_CD4
59193     Imm_other
59194     Imm_other
59195     Imm_other
            ...    
60524     Imm_other
60525     Imm_other
60526    Fibroblast
60527     Imm_other
60528     Imm_other
Name: Cluster, Length: 1338, dtype: category
Categories (7, object): ['Endothelial', 'Epithelial', 'Fibroblast', 'Imm_other', 'Myeloid_CD68', 'Tcell_CD4', 'Tcell_CD8']